In [1]:
from __future__ import print_function, division
from builtins import range

In [4]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
#from gensim.models import KeyedVectors


In [7]:
train = "C://Users//SONY//Desktop//WORK//PROJECT//train_chunks.xlsx"
test = "C://Users//SONY//Desktop//WORK//PROJECT//test_chunks.xlsx"

In [8]:

df = pd.read_excel(train)
df=df.dropna(axis=0)
df.Text = df.Text.str.replace('[^\w\s]','')
df.Text = df.Text.str.lower()
#df.Text = df.Text.str.split()

df1 = pd.read_excel(test)
df1=df1.dropna(axis=0)
df1.Text = df1.Text.str.replace('[^\w\s]','')
df1.Text = df1.Text.str.lower()
#df1.Text = df1.Text.str.split()

## TACKLING WITH UNBALANCED DATASET

In [9]:
len(df[df['Labels']==1])

200

In [10]:
A= df[df['Labels']==1]
B= df[df['Labels']==0]

In [11]:
A_repeated = pd.concat([A]*7, ignore_index=True)
df = pd.concat([A_repeated,B])
from sklearn.utils import shuffle
df = shuffle(df)
df = df.reset_index(drop=True)

In [12]:
df

,Text,Labels,Id
0,ya itd be bs if it wasnt balanced like tha...,0,subject2419
1,thanks everyone for the kind words33 i had...,1,subject6681
2,dat hair roto your sister might like t...,0,subject3763
3,in what context i have been getting ...,1,subject6446
4,lol simple mistake call them tha...,0,subject3788
...,...,...,...
2714,do you sometimes not cross the ts as well,1,subject3883
2715,you can eat as much as you wish in front o...,0,subject2840
2716,if it wasnt pink i would cop this nice fin...,0,subject3498
2717,adorable little derpface d i love him hes ...,1,subject1913


## FURTHER PRE-PROCESSING

In [13]:
#from nltk.corpus import stopwords 
#from nltk.tokenize import word_tokenize 

In [115]:
#stop = stopwords.words('english')
#df['Text'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [116]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [117]:
df['Text'] = df.Text.apply(lemmatize_text)

In [118]:
#df1['Text'] = df1['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df1['Text'] = df1.Text.apply(lemmatize_text)

## LOADING GLOVE VECTORS

In [55]:
class GloveVectorizer:
    def __init__(self):
        # load in pre-trained word vectors
        print('Loading word vectors...')
        word2vec = {}
        embedding = []
        idx2word = []
        with open('C:\\Users\\SONY\\Desktop\\WORK\\New\\glove\\glove.6B.100d.txt',encoding='utf') as f:
            # is just a space-separated text file in the format:
            # word vec[0] vec[1] vec[2] ...
            for line in f:
                values = line.split()
                word = values[0]
                vec = np.asarray(values[1:], dtype='float32')
                word2vec[word] = vec
                embedding.append(vec)
                idx2word.append(word)
        print('Found %s word vectors.' % len(word2vec))

        # save for later
        self.word2vec = word2vec
        self.embedding = np.array(embedding)
        self.word2idx = {v:k for k,v in enumerate(idx2word)}
        self.V, self.D = self.embedding.shape

    def fit(self, data):
        pass

    def transform(self, data):
        X = np.zeros((len(data), self.D))
        Y = np.zeros((len(data), self.D))
        Z = np.zeros((len(data), self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            #tokens = sentence.lower().split()
            vecs = []
            for word in sentence:
                if word in self.word2vec:
                    vec = self.word2vec[word]
                    vecs.append(vec)
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
                Y[n] = vecs.min(axis=0)
                Z[n] = vecs.max(axis=0)
                
            else:
                emptycount += 1
            n += 1
        print("Number of samples with no words found: %s / %s" % (emptycount, len(data)))
        return X,Y,Z

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [56]:
vectorizer = GloveVectorizer()
# vectorizer = Word2VecVectorizer()
Xavg,Xmin,Xmax = vectorizer.fit_transform(df.Text)
Ytrain = df.Labels
Test_avg,Test_min,Test_max = vectorizer.transform(df1.Text)
Ytest = df1.Labels

Loading word vectors...
Found 400000 word vectors.
Number of samples with no words found: 0 / 2719
Number of samples with no words found: 2 / 3198


In [57]:
Xavg.shape, Xavg

((2719, 100), array([[-0.103516  ,  0.074485  ,  0.22089323, ..., -0.17393164,
          0.41592377, -0.05083651],
        [-0.06503699,  0.20887189,  0.3932212 , ..., -0.26896009,
          0.39756155,  0.21378291],
        [-0.02464631,  0.19230519,  0.41903526, ..., -0.28894946,
          0.48738915,  0.20868236],
        ...,
        [-0.14407238,  0.19350828,  0.43613529, ..., -0.32242751,
          0.25936514,  0.2879793 ],
        [-0.14042388,  0.17408687,  0.24441531, ..., -0.26558679,
          0.44454801,  0.20874248],
        [-0.1039901 ,  0.23156282,  0.32641757, ..., -0.20961121,
          0.33832663,  0.24124797]]))

## TRYING SIMPLER ML MODELS

In [58]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear',class_weight='balanced',random_state=5,tol=0.001,max_iter=1000000)

In [59]:
#model = RandomForestClassifier(n_estimators=200)
model.fit(Xavg, Ytrain)
print("train score:", model.score(Xavg, Ytrain))
print("test score:", model.score(Test_avg, Ytest))


train score: 0.8444280985656492
test score: 0.8542839274546592


In [60]:
predictions = model.predict(Test_avg)

In [61]:
from sklearn.metrics import precision_score, recall_score,accuracy_score, classification_report, f1_score ,confusion_matrix


In [62]:
cm = confusion_matrix(Ytest,predictions)
print(cm)
print('\n',classification_report(Ytest,predictions))


[[2419  369]
 [  97  313]]

               precision    recall  f1-score   support

           0       0.96      0.87      0.91      2788
           1       0.46      0.76      0.57       410

    accuracy                           0.85      3198
   macro avg       0.71      0.82      0.74      3198
weighted avg       0.90      0.85      0.87      3198



In [63]:
model.fit(Xmax, Ytrain)
predictions2 = model.predict(Test_max)
cm2 = confusion_matrix(Ytest,predictions2)
print(cm2)
print('\n',classification_report(Ytest,predictions2))

[[2241  547]
 [ 158  252]]

               precision    recall  f1-score   support

           0       0.93      0.80      0.86      2788
           1       0.32      0.61      0.42       410

    accuracy                           0.78      3198
   macro avg       0.62      0.71      0.64      3198
weighted avg       0.85      0.78      0.81      3198



In [64]:
model.fit(Xmin, Ytrain)
predictions3 = model.predict(Test_min)
cm3 = confusion_matrix(Ytest,predictions3)
print(cm3)
print('\n',classification_report(Ytest,predictions3))

[[2213  575]
 [ 137  273]]

               precision    recall  f1-score   support

           0       0.94      0.79      0.86      2788
           1       0.32      0.67      0.43       410

    accuracy                           0.78      3198
   macro avg       0.63      0.73      0.65      3198
weighted avg       0.86      0.78      0.81      3198



In [65]:
from sklearn import svm
SVMM = svm.LinearSVC(class_weight='balanced',verbose=0, random_state=None,max_iter=1000)  

In [66]:
SVMM.fit(Xavg, Ytrain)
predictions4 = SVMM.predict(Test_avg)
cm4 = confusion_matrix(Ytest,predictions4)
print("train score:", SVMM.score(Xavg, Ytrain))
print("test score:", SVMM.score(Test_avg, Ytest))
print('\n',cm4)
print('\n',classification_report(Ytest,predictions4))

train score: 0.8521515262964325
test score: 0.8564727954971857

 [[2436  352]
 [ 107  303]]

               precision    recall  f1-score   support

           0       0.96      0.87      0.91      2788
           1       0.46      0.74      0.57       410

    accuracy                           0.86      3198
   macro avg       0.71      0.81      0.74      3198
weighted avg       0.89      0.86      0.87      3198



# FINAL MODEL FOR CLASSIFICATION

In [67]:
import pickle
#import sklearn
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

In [68]:
train_x = np.dstack([Xavg,Xmin,Xmax])

In [69]:
train_x.shape

(2719, 100, 3)

In [71]:
train_x = train_x.reshape(2719, 1,100, 3)

In [72]:
test_x = np.dstack([Test_avg,Test_min,Test_max])

In [73]:
test_x.shape

(3198, 100, 3)

In [74]:
test_x = test_x.reshape(3198, 1,100, 3)

In [75]:
train_y = np.asarray(pd.get_dummies(Ytrain), dtype = np.int8)
test_y = np.asarray(pd.get_dummies(Ytest), dtype = np.int8)

In [76]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((2719, 1, 100, 3), (2719, 2), (3198, 1, 100, 3), (3198, 2))

## DEFINING HELPER FUNCTIONS

In [77]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_conv(x,kernel_size,num_channels,depth):
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(conv2d(x, weights),biases))
    
def apply_pooling(x,kernel_size,stride_size):
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')


## MODEL PARAMETERS

In [155]:
input_height = 1
input_width = 100
num_labels = 2
num_channels = 3

batch_size = 20
kernel_size = 5
nbr_kernels = 30
num_hidden = 128

learning_rate = 0.001
momentum=0.9
decay_rate = 0.0005
epsilon = 0.01
training_epochs = 30

total_batches = train_x.shape[0] // batch_size


## BUILDING THE LAYERS

In [156]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

c = apply_conv(X,kernel_size,num_channels,nbr_kernels)        #FILTER LAYER 1
p = apply_pooling(c,10,2)                                     #POOLING LAYER 1

shape = p.get_shape().as_list()
p_flat = tf.reshape(p, [-1, shape[1] * shape[2] * shape[3]])

W = weight_variable([shape[1] * shape[2] * nbr_kernels * num_channels, num_hidden])
b = bias_variable([num_hidden])
f = tf.nn.relu(tf.add(tf.matmul(p_flat, W),b))

W2 = weight_variable([num_hidden, num_hidden])
b2 = bias_variable([num_hidden])
f2 = tf.nn.relu(tf.add(tf.matmul(f, W2),b2))


out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
softmax_out = tf.nn.softmax(tf.matmul(f2, out_weights) + out_biases)



In [157]:
loss = -tf.reduce_sum(Y * tf.log(softmax_out))
#optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, decay= decay_rate , momentum= momentum,epsilon= epsilon, use_locking=False, centered=False, name='RMSProp').minimize(loss)
correct_prediction = tf.equal(tf.argmax(softmax_out,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## TRAINING

In [158]:
cost_history = np.empty(shape=[1],dtype=float)

with tf.Session() as session:
    tf.global_variables_initializer().run()
    #summary_writer = tf.summary.FileWriter('./tensor/logs', graph=tf.get_default_graph())
    saver = tf.train.Saver()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            #_, c,summary = session.run([optimizer, loss,merged],feed_dict={X: batch_x, Y : batch_y})
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)
            #summary_writer.add_summary(summary, epoch)
        print("Epoch: ",epoch," Training Loss: ",c," Training Accuracy: ",
              session.run(accuracy, feed_dict={X: train_x, Y: train_y}))
    
    print("Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y}))
    preds = session.run(softmax_out, feed_dict={X: test_x, Y: test_y})
    #save_path = saver.save(session, "/tmp/model.ckpt")
    #print("Model saved in path: %s" % save_path)

Epoch:  0  Training Loss:  15.010111  Training Accuracy:  0.77013606
Epoch:  1  Training Loss:  12.355821  Training Accuracy:  0.7513792
Epoch:  2  Training Loss:  10.102001  Training Accuracy:  0.78116953
Epoch:  3  Training Loss:  9.26745  Training Accuracy:  0.7907319
Epoch:  4  Training Loss:  6.3943596  Training Accuracy:  0.77969843
Epoch:  5  Training Loss:  6.6742225  Training Accuracy:  0.64950347
Epoch:  6  Training Loss:  10.638524  Training Accuracy:  0.81243104
Epoch:  7  Training Loss:  31.852676  Training Accuracy:  0.53438765
Epoch:  8  Training Loss:  11.296098  Training Accuracy:  0.7833762
Epoch:  9  Training Loss:  7.7995186  Training Accuracy:  0.6811328
Epoch:  10  Training Loss:  6.490534  Training Accuracy:  0.7035675
Epoch:  11  Training Loss:  9.471545  Training Accuracy:  0.62522984
Epoch:  12  Training Loss:  4.741336  Training Accuracy:  0.8584038
Epoch:  13  Training Loss:  3.6864538  Training Accuracy:  0.78926075
Epoch:  14  Training Loss:  15.942212  Tr

In [159]:
preds.shape

(3198, 2)

In [160]:
preds

array([[5.6843599e-03, 9.9431568e-01],
       [2.4683070e-05, 9.9997532e-01],
       [4.0279412e-01, 5.9720594e-01],
       ...,
       [6.5125544e-03, 9.9348748e-01],
       [9.9660599e-01, 3.3940000e-03],
       [2.8382918e-01, 7.1617085e-01]], dtype=float32)

In [161]:
final_pred=[]
for i in preds:
    final_pred.append(np.argmax(i))

In [162]:
cm5 = confusion_matrix(Ytest,final_pred)
print('\n',cm5)
print('\n',classification_report(Ytest,final_pred))


 [[2183  605]
 [  91  319]]

               precision    recall  f1-score   support

           0       0.96      0.78      0.86      2788
           1       0.35      0.78      0.48       410

    accuracy                           0.78      3198
   macro avg       0.65      0.78      0.67      3198
weighted avg       0.88      0.78      0.81      3198

